In [1]:
import numpy as np
import matplotlib.pyplot as plt
from astroquery.sdss import SDSS
from urllib.parse import urlencode
from urllib.request import urlretrieve
import pandas as pd
from pathlib import Path
import os
import sqlite3


In [ ]:
with open('sql_query.sql', 'r') as f:
    sql_code = f.read()

sql_table=SDSS.query_sql(sql_code).to_pandas()
sql_table.to_csv(output_image.as_posix()+"/"+ "sqltable_galaxy.csv", sep=",")

In [13]:
#make a dir at the path
current_dir=Path(os.getcwd())
output_image = current_dir.joinpath("project2")
output_image.mkdir(parents=True, exist_ok=True)

In [12]:
sdss_query= """
      select 
      top 100 *
      from SpecPhoto
      where class='galaxy' and subClass='STARFORMING' and z<1
      """
sql_table=SDSS.query_sql(sdss_query).to_pandas()
sql_table.to_csv(output_image.as_posix()+"/"+ "sqltable.csv", sep=",")

In [8]:


#Image downloader
img_cutoutbaseurl = "https://skyserver.sdss.org/dr18/SkyServerWS/ImgCutout/getjpeg?TaskName=Skyserver.Explore.Image"
for ra, dec, objclass, spobjid in zip(sql_table['ra'], sql_table['dec'], 
                                      sql_table['class'], sql_table['specObjID']):
                                         
    img_query_string = urlencode(dict(ra=ra,
                                  dec=dec,
                                  width=500,height=500,
                                  scale=0.2, opt="GL"))
                                         

    #The url of the image
    img_url = img_cutoutbaseurl + '&' + img_query_string
    #Prep the name of the image and spec
    image_name = output_image.as_posix() + '/'+'image_{0}ra_{1}dec_{2}_{3}specObjID_SDSS_dr18.jpg'.format(ra,dec,objclass, spobjid)
    #Download the image and spectra
    urlretrieve(img_url, image_name)


In [10]:
#spectra downloader
sp_base='https://skyserver.sdss.org/dr18/en/get/SpecById.ashx'
for ra, dec, objclass, spobjid in zip(sql_table['ra'], sql_table['dec'], sql_table['class'],sql_table['specObjID']):
    sp_query_string = urlencode(dict(id=spobjid))
    sp_url=sp_base+'?'+sp_query_string
    sp_name=output_image.as_posix()+ '/'+'spectra_ra_{0}_dec_{1}_class_{2}_specobjid_{3}SDSS_dr18.png'.format(ra, dec, objclass, spobjid)
    urlretrieve(sp_url, sp_name)
    
    
    

In [2]:
#table to latext converter

In [2]:
import os

folder_path = 'project1/images/'  # Replace this with the path to your image folder
output_file = 'output.tex'  # Replace this with the desired output file name

# Get a list of all image files in the folder
image_files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]

# Open the output file for writing
with open(output_file, 'w') as f:
    # Write the LaTeX code for the figure
    f.write(r'\begin{figure}[h]' + '\n')
    f.write(r'\centering' + '\n')
    f.write(r'\begin{tabular}{cccc}' + '\n')

    # Iterate over the image files and write the LaTeX code for each image
    for i, image_file in enumerate(image_files):
        # Write the LaTeX code for the current image
        f.write(r'\includegraphics[width = 1.5in]{' + os.path.join(folder_path, image_file) + r'} &')

        # Start a new row after every 4 images
        if (i + 1) % 4 == 0:
            f.write(r'\\' + '\n')

    # Close the last row and finish the LaTeX code for the figure
    f.write(r'\end{tabular}' + '\n')
    f.write(r'\caption{4 x 4}' + '\n')
    f.write(r'\end{figure}' + '\n')

print(f'LaTeX code has been written to {output_file}')


LaTeX code has been written to output.tex
